### IMPORTS

In [8]:
!pip install xgboost

In [29]:
import pickle

import pandas as pd
import numpy as np


from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer



from sklearn.preprocessing import OneHotEncoder, StandardScaler
import xgboost as xgb

### LOAD DATA

In [30]:
df = pd.read_csv("./data/Churn Modeling.csv")

In [31]:
df.shape

(10000, 14)

In [32]:
get_missing_values(df)

Series([], dtype: int64)

In [33]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [34]:
df = df.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [35]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [36]:
var_cat = ['Geography','Gender']

In [37]:
var_num = ['CreditScore','Age', 'Tenure', 'Balance','NumOfProducts','EstimatedSalary','HasCrCard','IsActiveMember']

In [38]:
var_target = ['Exited']

In [39]:
features = list(var_cat + var_num)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [41]:
features

['Geography',
 'Gender',
 'CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'EstimatedSalary',
 'HasCrCard',
 'IsActiveMember']

In [42]:
X = df[features]
y = df[var_target]

### Spliting Data

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [44]:
len(X), len(X_train), len(X_test), len(y_train), len(y_test)

(10000, 7500, 2500, 7500, 2500)

In [45]:
X_train

,Geography,Gender,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,HasCrCard,IsActiveMember
2967,Germany,Female,579,39,5,117833.30,3,5831.00,0,0
700,France,Female,750,32,5,0.00,2,95611.47,1,0
3481,Spain,Female,729,34,9,53299.96,2,42855.97,1,1
1621,Spain,Male,689,38,5,75075.14,1,8651.92,1,1
800,France,Male,605,52,7,0.00,2,173952.50,1,1
...,...,...,...,...,...,...,...,...,...,...
9225,Germany,Female,594,32,4,120074.97,2,162961.79,1,1
4859,Spain,Female,794,22,4,114440.24,1,107753.07,1,1
3264,France,Male,738,35,5,161274.05,2,181429.87,1,0
9845,Spain,Female,590,38,9,0.00,2,148750.16,1,1


### Pipeline

In [46]:
OHE = OneHotEncoder()
scaler = StandardScaler()
XGB = xgb.XGBClassifier(max_depth=2, eta= 0.35)


transformer = ColumnTransformer([('cat_cols', OHE, var_cat),
                                ('num_cols', scaler, var_num)])

pipe = Pipeline([("preprocessing", transformer),
                ("classifier", XGB)])

pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat_cols', OneHotEncoder(),
                                                  ['Geography', 'Gender']),
                                                 ('num_cols', StandardScaler(),
                                                  ['CreditScore', 'Age',
                                                   'Tenure', 'Balance',
                                                   'NumOfProducts',
                                                   'EstimatedSalary',
                                                   'HasCrCard',
                                                   'IsActiveMember'])])),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, c...
                               eval_metric=None, gamma=0, gpu_id=-1,
                               grow_policy='depthwise', importance_type=

In [47]:
X_test.columns

Index(['Geography', 'Gender', 'CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'EstimatedSalary', 'HasCrCard', 'IsActiveMember'],
      dtype='object')

In [48]:
predicoes = pipe.predict(X_test)
predicoes[:10]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1])

In [49]:
y_test.values[:10].flatten()

array([0, 1, 0, 0, 0, 1, 0, 0, 1, 1], dtype=int64)

In [50]:
accuracy_score(y_test, predicoes)

0.8656

In [54]:
pickle.dump(pipe, open('./models/pipe.pkl','wb'))